In [0]:
%run "/Workspace/Shared/Team 4_2/flight-departure-delay-predictive-modeling/notebooks/Cross Validator/Cross Validator Module"

In [0]:
data_loader = FlightDelayDataLoader()
data_loader.load()

In [0]:
folds = data_loader.get_version("3M")

In [0]:
folds